In [4]:
import os
import random 

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm
from functools import partial
import math
# import heapq
from scipy.io import wavfile
from IPython import display

import matplotlib.pyplot as plt 
from torchaudio.transforms import MelScale, Spectrogram, MelSpectrogram, InverseMelScale
from torchaudio.functional import istft, griffinlim
import torchaudio
import librosa

import sys
sys.path.append("/home/zakirov/repoz/voice-transfer/")


from voice_transfer.audio import melspectrogram2wav

# %load_ext autoreload
# %autoreload 2

In [ ]:
#Hyperparameters

params = {
#     "data_path" : "datasets/cmu_arctic",
    "hop_size" : 256,
    "window_size" : 1024,
    "n_fft" : 1024,
    "n_mels" : 80,
    "sampling_rate" : 22050,
#     "duration" : 10.0,
    "ref_level_db" : 20.,
    "min_level_db" : -80.0, #-80
    "train_size" : 0.8,
    "f_min" : 0.0,
    "f_max" : None,
}

In [5]:
from voice_transfer.datasets import AudioMelDataset, get_dataloader, AudioDataset
import torchvision
from voice_transfer.modules import RandomCrop1D
from torch.utils.data import Dataset, DataLoader, random_split
from functools import reduce

In [ ]:
transforms = torchvision.transforms.Compose([
    RandomCrop1D(size=50000),
#     torchaudio.transforms.Fade(fade_in_len=10000, fade_out_len=10000, fade_shape='linear')
])

dataset = AudioDataset("datasets/cmu_arctic", transforms, resample=True, sampling_rate=16000)
# dataset = dataset_class("datasets/LJSpeech-1.1", transforms, resample, sampling_rate, **kwargs)

#     if "vctk" in datasets:
#         dataset = dataset_class("datasets/VCTK-Corpus", transforms, resample, sampling_rate, **kwargs)
all_datasets=[]
train_size=0.8
train_length=int(len(dataset) * train_size)
val_length=len(dataset) - train_length
train_d, val_d = random_split(dataset, (train_length, val_length))
all_datasets.append(train_d)
 
# Concat all datasets into one
all_datasets = reduce(lambda x, y: x + y, all_datasets)

# without `drop_last` last batch consists of 1 element and BN fails TODO(zakajd): Remove this ???

dataloader = DataLoader(
    all_datasets, 
    batch_size=64, 
    shuffle=True, 
    num_workers=8, 
    drop_last=True, 
    pin_memory=True)

In [ ]:
transforms = torchvision.transforms.Compose([
    RandomCrop1D(size=50000),
#     torchaudio.transforms.Fade(fade_in_len=10000, fade_out_len=10000, fade_shape='linear')
])
dataloader = get_dataloader(["arctic"], transforms=transforms)

In [ ]:
waveform = dataset[1]
# waveform = torch.ones((1, 50000)) - torch.randint(0, 2, (1, 50000))
plt.plot(waveform.squeeze().numpy())

In [ ]:
# "quarter_sine", "half_sine", "linear", "logarithmic", "exponential"
    
fade = torchaudio.transforms.Fade(fade_in_len=2000, fade_out_len=2000, fade_shape='linear')

In [ ]:
faded = fade(waveform)
plt.plot(faded.squeeze().numpy())

In [ ]:
display.display(display.Audio(waveform.numpy(), rate=params["sampling_rate"]))
display.display(display.Audio(faded.numpy(), rate=params["sampling_rate"]))

In [ ]:
file_path = "datasets/cmu_arctic/cmu_us_bdl_arctic/wav/"
file_path_other = "datasets/cmu_arctic/cmu_us_clb_arctic/wav/"

file_name = random.choice(os.listdir(file_path))
print(file_path + file_name)
display.Audio(filename=file_path + file_name)

In [ ]:
print(file_path_other + file_name)
display.Audio(filename=file_path_other + file_name)

In [ ]:
waveform, sample_rate = torchaudio.load(file_path + file_name, )
print("Shape of waveform: {}".format(waveform.size()))
print("Sample rate of waveform: {}".format(sample_rate))
length = waveform.size()[1]
plt.figure()
plt.plot(waveform.t().numpy())

'/home/zakirov/repoz/voice-transfer'

In [3]:
from voice_transfer.audio import melspectrogram2wav